### Data Science Challenge: Card Transactions


#### Question 1: Load

-	1.1 Programmatically load the data into your favorite analytical tool. 
-	1.2 Please describe the structure of the data. Number of records and fields in each record?
-	1.3 Please provide some additional basic summary statistics for each field. Be sure to include a count of null, minimum, maximum, and unique values where appropriate.


#### Question 2: Plot

-	2.1 Plot a histogram of the processed amounts of each transaction, the transactionAmount column. 
-   2.2 Report any structure you find and any hypotheses you have about that structure.


#### Question 3: Data Wrangling - Duplicate Transactions
You will notice a number of what look like duplicated transactions in the data set. One type of duplicated transaction is a reversed transaction, where a purchase is followed by a reversal. Another example is a multi-swipe, where a vendor accidentally charges a customer's card multiple times within a short time span.

-	3.1 Can you programmatically identify reversed and multi-swipe transactions?
-	3.2 What total number of transactions and total dollar amount do you estimate for the reversed transactions? For the multi-swipe transactions? (please consider the first transaction to be "normal" and exclude it from the number of transaction and dollar amount counts)
-	3.3 Did you find anything interesting about either kind of transaction?


#### Question 4: Model

Fraud is a problem for any bank. Fraud can take many forms, whether it is someone stealing a single credit card, to large batches of stolen credit card numbers being used on the web, or even a mass compromise of credit card numbers stolen from a merchant via tools like credit card skimming devices.
-	4.1 Each of the transactions in the dataset has a field called isFraud. Please build a predictive model to determine whether a given transaction will be fraudulent or not. Use as much of the data as you like (or all of it).
-	4.2 Provide an estimate of performance using an appropriate sample, and show your work.
-	4.3 Please explain your methodology (modeling algorithm/method used and why, what features/data you found useful, what questions you have, and what you would do next with more time)


#### The following artifacts are valued:
-	explanations of your intent, methods, conclusions and any assumptions
-	clear, documented, and well-structured code
-	instructions for running your code
-	methods you attempted that didn't work
-	ideas you didn't have time to complete but would have done with more time
-	a thorough write up with any pertinent visualizations


##### How to run this code:


I) set up this environment:


conda create -n xgboost_env -c conda-forge python=3 numpy pandas scikit-learn category_encoders ipython-notebook wget matplotlib=3 xgboost unzip 

source activate xgboost_env




II) launch the notebook:

ipython notebook card_transactions_takehome.ipynb


III) run all cells

In [ ]:
# load libs and check if versions are correct
import os
import sys
import time
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import category_encoders as ce
import xgboost
from xgboost import XGBClassifier
from xgboost import plot_importance

print("MacBook Air Early 2014; OS 10.14.6")
print("The Python version is %s.%s.%s" % sys.version_info[:3])
print('The numpy version is {}.'.format(np.__version__))
print('The pandas version is {}.'.format(pd.__version__))
print('The matplotlib version is {}.'.format(matplotlib.__version__))
print('The scikit-learn version is {}.'.format(sklearn.__version__))
print('The category_encoders version is {}.'.format(ce.__version__))
print('The xgboost version is {}.'.format(xgboost.__version__))

#The Python version is 3.5.5
#The numpy version is 1.15.2.
#The pandas version is 0.23.4.
#The matplotlib version is 3.0.0.
#The scikit-learn version is 0.20.0.
#The category_encoders version is 2.1.0.
#The xgboost version is 0.80.

In [ ]:
# FUNCTIONS

In [ ]:
# functions to investigate df

In [ ]:
# function to look at data frame shape, count NA per field
def df_shape_nacount(df):
    shape = df.shape 
    print('shape')
    print(shape)
    print('')

    fields = list(df) 
    # count NAs 
    # treat inf as Na
    pd.options.mode.use_inf_as_na = True
    found_one = 0
    for i in fields:  
        if(sum(df[i].isna()))>0:
            print (i)
            print("Na count: ")
            print (sum(df[i].isna()))  
            print('')
            found_one = 1
    if(found_one ==0):
        print('no NA found at all') 

In [ ]:
# function to look at data frame shape, count and list unique values per field
def df_shape_unique_countlist(df):
    shape = df.shape 
    print('shape')
    print(shape)
    print('')

    fields = list(df) 
    # count and list unique values per field
    for i in fields:  
        print (i) 
        print ('unique count: ') 
        uniq = pd.unique(df[i])
        print(uniq.shape[0])
        print ('unique list') 
        print (uniq)
        print ('') 

In [ ]:
# function to look at data frame shape, count None per category field  
def df_shape_nonecount(df):
    shape = df.shape 
    print('shape')
    print(shape)
    print('')

    fields = list(df) 
    # count and list ''  per field
    for i in fields:
        if(df[i].dtypes == 'O' and (sum(df[i] == '') > 0)):    
            print (i) 
            print ('None count: ') 
            print (sum(df[i] == ''))
            print('')

In [ ]:
# model-related functions

In [ ]:
# function to initialize and fit xgboost with a given 'scale_pos_weight_value'
# this function also calls the accuracy calculation function 
# and the function to calculate the net $ result of applying the fitted model
def trial_xgboost_scale_pos_weight(scale_pos_weight_value, x_var_train, x_var_eval, y_var_eval, transfeefrac):
    # define model
    model = XGBClassifier(scale_pos_weight=scale_pos_weight_value, seed=1)
    # fit model
    model.fit(X = x_var_train, y = label_train_encoded_y)
    # predict
    y_pred = model.predict(x_var_eval)
    
    print("scale_pos_weight used: ",  scale_pos_weight_value)
    transamount = x_var_eval['transactionAmount']
    # call function to calculate accuracy and cost
    calc_accurcy(y_var_eval, y_pred)
    # call function to calculate $ net
    net = calc_cost(y_var_eval, y_pred, transamount, transfeefrac)
    # return the fitted model and the $ net
    return model, net

In [ ]:
# function to calculate the accuracy values of a model 
# results are printed to screen
def calc_accurcy(y, y_hat):
    #count cases
    sample_count = len(y)
    fraud_count = sum(y)
    fraud_pred_count = sum(y_hat)
    
    correct_fraud_pred_count = sum((y==1) & (y_hat==1))
    correct_nofraud_pred_count = sum((y==0) & (y_hat==0))
    incorrect_fraud_pred_count = sum((y==0) & (y_hat==1)) #false_fraud
    incorrect_nofraud_pred_count = sum((y==1) & (y_hat==0)) #missed_fraud
    print('count all frauds: ', fraud_count)
    print('count correct fraud predictions: ', correct_fraud_pred_count)
    print('count false fraud predictions: ', incorrect_fraud_pred_count)
    print('count missed frauds: ', incorrect_nofraud_pred_count)

    # calculate overall accuracy, precision, recall
    # p is correct positive results divided by all positive results returned by the classifier. 
    p = correct_fraud_pred_count / fraud_pred_count
    # r is correct positive results divided by all samples that should have been identified as positive. 
    r = correct_fraud_pred_count / fraud_count
    # overall accuracy
    o = (correct_fraud_pred_count + correct_nofraud_pred_count) /sample_count

    # print overall accuracy, precision, recall w 5 d.p.
    print('overall accuracy: ',round(o,3), 'precision: ', round(p,3), 'recall: ',round(r,3))
    print('precision is the ability of the classifier not to label as positive a sample that is negative.')
    print('The recall is the ability of the classifier to find all the positive samples')

In [ ]:
# function to calculate the net $ result of applying the model
def calc_cost(y,  y_hat, transamount, transfeefrac, benchmark = False):
    if(benchmark):
        # calculate w/o a fraud detection model:
        # missed fraud array
        missed_fraud = (y==1) # use all fraud
        # cost is the sum of transamounts * missed_fraud cases (here = all fraud)
        cost = round(sum(transamount*missed_fraud))
        # income is the sum of transamounts * transfeefrac 
        income = round(sum(transamount*transfeefrac))    
    else: 
        # calculate using the fraud model results provided:
        # missed fraud array
        missed_fraud = (y==1) & (y_hat==0)
        # cost is the sum of transamounts * missed_fraud cases (zeros and ones)
        cost = round(sum(transamount*missed_fraud))
        # income is the sum of transamounts * transfeefrac * cases not flagged as fraud (zeros and ones)
        income = round(sum(transamount*transfeefrac*(1-y_hat)))
        
    # $ net is income - cost
    net = income - cost
    #
    print('---')
    print('income $: ', income) 
    print('cost $: ', cost)
    print('net $: ', net) 
    print('---')
    # $ net is returned
    return net

#### Question 1: Load

-	1.1 Programmatically load the data into your favorite analytical tool the transactions data. 
-	1.2 Please describe the structure of the data. Number of records and fields in each record?
-	1.3 Please provide some additional basic summary statistics for each field. Be sure to include a count of null, minimum, maximum, and unique values where appropriate.

##### 1.1 Programmatically load the data into  your favorite analytical tool the transactions data. 

In [ ]:
# unzip and rm what is no longer needed
com = "unzip transactions.txt.zip"
os.system(com)
com = "rm transactions.txt.zip"
os.system(com)

In [ ]:
# read the data into pd
df = pd.read_json('transactions.txt', lines=True)

##### 1.2 Please describe the structure of the data. Number of records and fields in each record?

In [ ]:
# check dimensions: 29 fields
print('number of records, number of fields: ', df.shape)

In [ ]:
# preserve the origional_index in a field called 'origional_index'
df['origional_index'] = df.index

##### 1.3 Please provide some additional basic summary statistics for each field. Be sure to include a count of null, minimum, maximum, and unique values where appropriate.

In [ ]:
# take a peek at the data
df[0:3]

In [ ]:
# list all fields as we cannot see them in the field above
fields = list(df)
fields

In [ ]:
# check for NA values in all fields
df_shape_nacount(df)

In [ ]:
# get counts of unique values per field and show values
df_shape_unique_countlist(df)

In [ ]:
# drop fields that have only one value as not useful
df.drop(['echoBuffer', 'merchantCity', 'merchantState', 'merchantZip', 'posOnPremises', 'recurringAuthInd'], axis='columns', inplace=True)
# number of fields I am left with: 
df.shape[1]

In [ ]:
#check non numeric files for missing entries (=='') and count the missing entries’
df_shape_nonecount(df)

In [ ]:
# I note that there are cases where the transactionType is not known

In [ ]:
# I do not comprehend the following fields and could not ask for clarification
# I hence drop them from further analysis as using fields that I do not understand in transaction 
# modeling would be risky
df.drop(['posConditionCode', 'posEntryMode'], axis='columns', inplace=True)
# number of fields I am left with: 
df.shape[1]

In [ ]:
# test if 'accountNumber' = 'customerId'
df[df['accountNumber'] == df['customerId']].shape[0] == df.shape[0]

In [ ]:
# drop customerId as it is a duplicate of accountNumber
df.drop(['customerId'], axis='columns', inplace=True)
# number of fields I am left with: 
df.shape[1]

In [ ]:
# describing fields as summary stats

In [ ]:
#categroical fields
df.describe(include='O') 

In [ ]:
#bool fields
df.describe(include='bool') 

In [ ]:
#numerical fields
df.describe(include=np.number) 

#### Question 2: Plot

-	2.1 Plot a histogram of the processed amounts of each transaction, the transactionAmount column. 
-   2.2 Report any structure you find and any hypotheses you have about that structure.


#####  2.1 Plot a histogram of the processed amounts of each transaction, the transactionAmount column. 

In [ ]:
df['transactionAmount'].hist(bins=100)

the distribution is dominated by small transactions and counter per bin decrease with transaction size

In [ ]:
# I separate fraud from nofraud and count the respective occurrences
fraud = df[df['isFraud']==1]
nofraud = df[df['isFraud']==0]

print('fraud count: ', fraud.shape[0])
print('nofraud count: ', nofraud.shape[0])
print('all sample count: ', df.shape[0])
print('Fraud is rare. Only  ', round(fraud.shape[0]/df.shape[0] * 100, 2), ' % of all samples')

Fraud is rare

In [ ]:
# comparing the histogram of transaction amounts of the entire sample with ....
nbins = 100
xmin = 0
xmax = 1000

fig = plt.figure(); 
ax = fig.add_subplot(1, 1, 1)
ax.hist(df['transactionAmount'], bins=nbins,range=[xmin,xmax])  
plt.show()

In [ ]:
# ... the histogram of transaction amounts of fraud and with ...
fig = plt.figure(); 
ax = fig.add_subplot(1, 1, 1)
ax.hist(fraud['transactionAmount'], bins=nbins,range=[xmin,xmax])  
plt.show()

In [ ]:
# ... the histogram of transaction amounts of nofraud 
fig = plt.figure(); 
ax = fig.add_subplot(1, 1, 1)
ax.hist(nofraud['transactionAmount'], bins=nbins,range=[xmin,xmax])  
plt.show()

Fraud appears to have larger transaction amount than nofraud

In [ ]:
# same comparison of transaction amount in all samples vs ...
df['transactionAmount'].describe() 

In [ ]:
# transaction amount of fraud samples vs ...
fraud['transactionAmount'].describe() 

In [ ]:
# transaction amount of no fraud samples 
nofraud['transactionAmount'].describe() 

#####  2.2 Report any structure you find and any hypotheses you have about that structure.

Summary: 
- The distribution of transactionAmount is dominated by small transactions and counter per bin decrease with transaction size
- Fraud appears to have larger transaction amount (e.g. higher median)
- Hypotheses: the transactionAmount itself could be a useful feature for predicting fraud

#### Question 3: Data Wrangling - Duplicate Transactions
You will notice a number of what look like duplicated transactions in the data set. One type of duplicated transaction is a reversed transaction, where a purchase is followed by a reversal. Another example is a multi-swipe, where a vendor accidentally charges a customer's card multiple times within a short time span.

-	3.1 Can you programmatically identify reversed and multi-swipe transactions?
-	3.2 What total number of transactions and total dollar amount do you estimate for the reversed transactions? For the multi-swipe transactions? (please consider the first transaction to be "normal" and exclude it from the number of transaction and dollar amount counts)
-	3.3 Did you find anything interesting about either kind of transaction?


In [ ]:
# filter out address verifications (assuming that there are no omission or commission errors in this field)
df_subset = df[df['transactionType'] != 'ADDRESS_VERIFICATION']
# number of samples I am left with: 
df_subset.shape[0]

##### 3.1a Can you programmatically identify reversed transactions?
##### 3.2a What total number of transactions and total dollar amount do you estimate for the reversed transactions? 

In [ ]:
# programmatically identify reversed:

# reversed transactions can be easily identified the REVERSAL flag in the transactionType field
# (assuming that there are no omission or commission errors of this field)
df_reversed = df_subset[df_subset ['transactionType'] == 'REVERSAL']

print('the number of reversed transaction is :', df_reversed.shape[0])
print('the total transaction amount of the reversed transaction is $:', round(sum(df_reversed['transactionAmount'])))

# I can link the result back to the df using the 'origional_index' field

In [ ]:
# counting the number of reversed transactions e.g. per merchantCategoryCode
df_reversed['merchantCategoryCode'].value_counts()

Online_retail followed by fastfood dominates merchantCategoryCode of the reversed transactions

##### 3.1b Can you programmatically identify multi-swipe transactions?

In [ ]:
# drop the reversed transactions from the subset df
df_subset = df_subset[df_subset ['transactionType'] != 'REVERSAL']
# number of samples I am left with: 
df_subset.shape[0]

In [ ]:
# I am looking for multiple occurrences of the same 'event' (once ADDRESS_VERIFICATION and REVERSAL has been taken out)

# assuming that an event must involve multiple occurrences of identical combinations of:
# 'accountNumber', 'cardLast4Digits', 'cardCVV', 'merchantCategoryCode','merchantCountryCode', 'merchantName'
# I do not use 'transactionAmount' as somebody may increase the amount for a repeat swipe
df_possible_multi_swipe = df_subset[df_subset.duplicated(['accountNumber', 'cardLast4Digits', 'cardCVV','merchantCategoryCode', 'merchantCountryCode', 'merchantName'], keep = False)]
del(df_subset)
# number of suspectsamples:
df_possible_multi_swipe.shape[0]

I am assuming no NA or missing values and no fake entries (a merchant can be identified as opposed to appear as somebody else)

In [ ]:
# I translate transactionDateTime into a pd datetime64 value and map it to a new field called: date_time'
transactionDateTime = df_possible_multi_swipe['transactionDateTime'].astype("datetime64")
df_possible_multi_swipe.insert(loc = df_possible_multi_swipe.shape[1], column = 'date_time', value = transactionDateTime)
del(transactionDateTime)

In [ ]:
# to ensure that the df is sorted I sort the suspect sample by  the following fields: 
# 'accountNumber', 'cardLast4Digits', 'cardCVV','merchantCategoryCode', 'merchantCountryCode', 'merchantName', 'date_time'
df_possible_multi_swipe = df_possible_multi_swipe.sort_values(by=['accountNumber', 'cardLast4Digits', 'cardCVV','merchantCategoryCode', 'merchantCountryCode', 'merchantName', 'date_time'])

I assume that multi swipe can only occur when card is present

In [ ]:
# assuming that multi swipe can only occur when card is present
df_possible_multi_swipe = df_possible_multi_swipe[df_possible_multi_swipe['cardPresent']==True]
# number of suspectsamples:
# counts dropped to ~1/3
df_possible_multi_swipe.shape[0]

In [ ]:
# I calculate the time difference between subsequent transactions in seconds 
delta_subsequent_enties_in_sec = (df_possible_multi_swipe['date_time']-df_possible_multi_swipe['date_time'].shift())/ np.timedelta64(1, 's')
# set the first entry to zero
delta_subsequent_enties_in_sec[0] = 0 

# I reset the indices of df_possible_multi_swipe and delta_subsequent_enties_in_sec so they 
# I can insert delta_subsequent_enties_in_sec values
df_possible_multi_swipe.index = pd.RangeIndex(len(df_possible_multi_swipe.index))
delta_subsequent_enties_in_sec.index = pd.RangeIndex(len(delta_subsequent_enties_in_sec.index))

# I map the time difference between subsequent transactions in seconds to a field called: 'delta'
df_possible_multi_swipe.insert(loc = df_possible_multi_swipe.shape[1], column = 'delta', value = delta_subsequent_enties_in_sec)

I define multi-swipe transactions as occurrences of identical combinations of: 'accountNumber', 'cardLast4Digits', 'cardCVV', 'merchantCategoryCode','merchantCountryCode', 'merchantName' within and 3 min

In [ ]:
# dif time delta > 0 and < 180 sec
mindif = 0
maxdif = 180 

In [ ]:
# subset > mindif and < maxdif
multi_swipe = df_possible_multi_swipe[(df_possible_multi_swipe['delta'] > mindif) & (df_possible_multi_swipe['delta'] < maxdif)]
del(df_possible_multi_swipe)

##### 3.2b What total number of transactions and total dollar amount do you estimate for the multi-swipe transactions?

In [ ]:
print('number of multi-swipe transactions found: ', multi_swipe.shape[0])
# I can link the back to the df using the 'origional_index' field

In [ ]:
print('Total dollar amount due to multi-swipe transactions: ', sum(multi_swipe['transactionAmount']))

##### 3.3 Did you find anything interesting about either kind of transaction?

In [ ]:
print('the fraction of multi-swipes flagged as fraud is: ', round(sum(multi_swipe['isFraud'])/multi_swipe.shape[0]*100,2), "%")

fraction of multi-swipes flagged as fraud did not appear unusually high....

In [ ]:
# Now comparing normalized histograms of transactionAmount for:
# the entire sample, fraud, nofraud, reversed transactions, and the multi swipe transactions

In [ ]:
# all transactions
fig = plt.figure(); 
ax = fig.add_subplot(1, 1, 1)
ax.hist(df['transactionAmount'], density = True, bins=nbins,range=[xmin,xmax])  
plt.show()

lots of very small $ amounts

In [ ]:
# fraud
fig = plt.figure(); 
ax = fig.add_subplot(1, 1, 1)
ax.hist(fraud['transactionAmount'], density = True, bins=nbins,range=[xmin,xmax])  
plt.show()

fraud peak at $100

In [ ]:
# nofraud 
fig = plt.figure(); 
ax = fig.add_subplot(1, 1, 1)
ax.hist(nofraud['transactionAmount'], density = True, bins=nbins,range=[xmin,xmax])  
plt.show()

nofraud similar to all samples (as expected given that fraud is rare)

In [ ]:
# reversed:  lots of very small amounts similar to all samples
fig = plt.figure(); 
ax = fig.add_subplot(1, 1, 1)
ax.hist(df_reversed['transactionAmount'], density = True, bins=nbins,range=[xmin,xmax])  
plt.show()

reversed has lots of very small amounts similar to all sample

In [ ]:
# multi_swipe has a second peak at ~$50 which is different from all samples and non-fraud
fig = plt.figure(); 
ax = fig.add_subplot(1, 1, 1)
ax.hist(multi_swipe['transactionAmount'], density = True, bins=nbins,range=[xmin,xmax])  
plt.show()

multi_swipe has a second peak at ~ $50 which is different from all sample and non-fraud

In [ ]:
# counting the number of counting e.g. per merchantCategoryCode
multi_swipe['merchantCategoryCode'].value_counts()

Now FAST FOOD dominates (previously it was online_retail); however the assumption that multi swipes can only occur when a card is present is risky as an online retail store can charge a card multiple times also....

In [ ]:
del(multi_swipe)
del(fraud)
del(nofraud)

#### Question 4: Model

Fraud is a problem for any bank. Fraud can take many forms, whether it is someone stealing a single credit card, to large batches of stolen credit card numbers being used on the web, or even a mass compromise of credit card numbers stolen from a merchant via tools like credit card skimming devices.
-	4.1 Each of the transactions in the dataset has a field called isFraud. Please build a predictive model to determine whether a given transaction will be fraudulent or not. Use as much of the data as you like (or all of it).
-	4.2 Provide an estimate of performance using an appropriate sample, and show your work.
-	4.3 Please explain your methodology (modeling algorithm/method used and why, what features/data you found useful, what questions you have, and what you would do next with more time)

##### 4.1 Each of the transactions in the dataset has a field called isFraud. Please build a predictive model to determine whether a given transaction will be fraudulent or not. Use as much of the data as you like (or all of it).

In [ ]:
# prepare the data for modeling

In [ ]:
# filter by transactionType ['PURCHASE' 'ADDRESS_VERIFICATION' 'REVERSAL' ''] and call the result: df_subset_fraud

# drop 'ADDRESS_VERIFICATION'
df_subset_fraud = df[df['transactionType'] != 'ADDRESS_VERIFICATION']
# drop 'REVERSAL'
df_subset_fraud = df_subset_fraud[df_subset_fraud ['transactionType'] != 'REVERSAL']

print('initial number of sample , number fields: ', df.shape)
print('number of sample , number fields left: ', df_subset_fraud.shape)
print('number of sample , number fields left: ', df_subset_fraud.shape)

In [ ]:
# list fields 
fields = list(df_subset_fraud)
fields

In [ ]:
# I drop a number of fields either because they may lead to overfitting (e.g. origional_index), 
# or they should not be a fraud predictor (e.g. cardLast4Digits),
# or because this simplifies my proof of concept model (e.g. enteredCVV or dateOfLastAddressChange), 
# or because there are too many categories for on hot encoding (e.g. merchantName)
df_subset_fraud.drop(['accountNumber', 'accountOpenDate', 'acqCountry', 'cardLast4Digits', 'cardCVV', 'currentExpDate', 'dateOfLastAddressChange', 'enteredCVV', 'merchantName','transactionDateTime', 'transactionType', 'origional_index'], axis='columns', inplace=True)
print('number fields left: ', df_subset_fraud.shape[1])

I dropped a number of fields either because they may lead to overfitting, or they should not be a fraud predictor, or because this simplifies my proof of concept model,  or because there are too many categories for on hot encoding 

In [ ]:
# list fields 
fields = list(df_subset_fraud)
fields

In [ ]:
# get counts of unique values per field and show values
df_shape_unique_countlist(df_subset_fraud)

In [ ]:
# check non numeric files for missing entries (=='') and count the missing entries’
df_shape_nonecount(df_subset_fraud)

only merchantCountryCode has '' values

In [ ]:
# select features for one hot encoding
df_fro_ohe = df_subset_fraud[['merchantCategoryCode', 'merchantCountryCode']]
print('number of sample , number fields : ', df_fro_ohe.shape)

In [ ]:
# drop those features from df_subset_fraud
df_subset_fraud.drop(['merchantCategoryCode', 'merchantCountryCode'], axis='columns', inplace=True)
print('number of sample , number fields left: ', df_subset_fraud.shape)

In [ ]:
# do the one hot encoding of the non-ordinal category variables
ohe = ce.OneHotEncoder(handle_unknown='ignore', use_cat_names=True)
x_train_ohe = ohe.fit_transform(df_fro_ohe)
del(ohe)

In [ ]:
# concat df_subset_fraud with the x_train_ohe (the one hot encoding)
df_subset_fraud = pd.concat([df_subset_fraud, x_train_ohe], axis=1, join='inner')

In [ ]:
# check the dims and lsit the fields 
print('number of sample , number fields : ', df_subset_fraud.shape)
fields = list(df_subset_fraud)
fields

In [ ]:
# take a peek at the data for modeling 
df_subset_fraud[0:3]

In [ ]:
# I split the data into 3 sets: a training set, a test set and a holdout set. 
# the purpose of the 3 sets is for training, hyperparameter tuning and final testing of the model, respectively
# the training set is 50% of the data and the test and holdout sets are 25% respectively

np.random.seed(1)
seed = 1
test_size = 0.5

# define the Y variable
Y = df_subset_fraud.iloc[:, 5]

# define the fieldscolumes for the X variables
li_1 =[0,1,2,3,4]
li_2 = list(range(6, df_subset_fraud.shape[1]))
joinedlist = li_1 + li_2
X = df_subset_fraud.iloc[:,joinedlist]

# break the dataset into set into a training and test set
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
# break the test set into a test and holdout set
x_test, x_holdout, y_test, y_holdout = train_test_split(x_test, y_test, test_size=test_size, random_state=seed)

# print the number of samples and fields in the various subsets 
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_holdout.shape)
print(y_holdout.shape)

del(df_subset_fraud)

In [ ]:
# encode the fraud variable for XGBClassifier
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(y_train)
label_train_encoded_y = label_encoder.transform(y_train)

del(label_encoder)

##### 4.2a Provide an estimate of performance using an appropriate sample, and show your work.

###### How to eval a fraud classifier (a model that can block fraud)? 

###### By considering net $ profit: 
calculating....

- 'income' as all transactions that were not blocked * 0.015 (assuming a 1.5% fee on transactions)
- 'cost' as the total value of fraud transactions that were not detected by the model
- 'net' as income - cost. 


$ Net is the amount we want to maximize


##### benchmark $ net when doing nothing... I do not try to block any fraud... on the test data

In [ ]:
# benchmark when doing nothing... I do not try to block any fraud... on the test data
transamount = x_test['transactionAmount']
transfeefrac = 0.015
y_no_model = ''
calc_cost(y_test, y_no_model, transamount, transfeefrac, benchmark = True)

So $393,062 would be earned if there was no fraud


But $660,087 had to be paid b/c we did not have a model in place to reject fraud

So we end up with a net loss of $267,025








Given that the fraud is rare we put more weight on fraud cases when training the xgboost model (or any model)


We incrementally increase the weight on fraud cases while keeping track of the $ net result of applying the model to the test data.


#### We keep the model with the largest $ net

In [ ]:
transfeefrac = 0.015

# loop over increasing 'scale_pos_weight' values
scale_pos_weight_list = [1,2,4,8,16,32,64,128]
for current_scale_pos_weight in scale_pos_weight_list:
    # fit the model with the current 'scale_pos_weight' value
    model, net = trial_xgboost_scale_pos_weight(current_scale_pos_weight, x_train, x_test, y_test, transfeefrac)
    #
    if(current_scale_pos_weight==1):
        best_model = model
        best_scale_pos_weight = current_scale_pos_weight
        best_net = net
    # compare if the current net is > than the highest one so far. 
    # if yes update the best model and net
    if(net > best_net):
        best_model = model
        best_scale_pos_weight = current_scale_pos_weight
        best_net = net
    #
print('the maximum net of $: ', round(best_net), ' was archived with a scale_pos_weight of: ', best_scale_pos_weight)
model = best_model

#### The model for the best model in terms of overall accuracy, precision, and recall is not very strong in terms of both precision but also recall. BUT it does earn the most $$

##### 4.2b Provide an estimate of performance using an appropriate sample, and show your work.

Now I run my chosen model on the holdout sample (that was not involved in model fitting or hyper parameter tuning)


##### benchmark $ net when doing nothing... I do not try to block any fraud... on the holdout sample

In [ ]:
# benchmark $ net when doing nothing... I do not try to block any fraud... on the holdout sample
# loss to of all fraud:

transamount = x_holdout['transactionAmount']
transfeefrac = 0.015
y_no_model = ''
calc_cost(y_holdout, y_no_model, transamount, transfeefrac, benchmark = True)

####
# so $393407 would be earned if there was no fraud
# but $674133 had to be paid b/c we did not have a model in place to reject fraud
# so we end up with a net loss of $280726

##### applying the model to the holdout sample

In [ ]:
# applying the model to the holdout sample
y_pred = model.predict(x_holdout)
transamount = x_holdout['transactionAmount']
transfeefrac = 0.015
calc_accurcy(y_holdout, y_pred)
net = calc_cost(y_holdout, y_pred, transamount, transfeefrac)
net

#####  the net when applying the model to the holdout sample is comparable to the net I got for the test sample  

### As opposed to a net loss of USD -280,726 the model results in a net profit (> USD 50,000)

In [ ]:
# plot feature importance

#importance_types
#‘weight’ - the number of times a feature is used to split the data across all trees.
#‘gain’ - the average gain across all splits the feature is used in.
#‘cover’ - the average coverage across all splits the feature is used in.
#‘total_gain’ - the total gain across all splits the feature is used in.
#‘total_cover’ - the total coverage across all splits the feature is used in.

# I use 'total_gain'

fig, ax = plt.subplots(figsize=(10, 10))
plot_importance(model, importance_type='total_gain', ax=ax)
plt.show()

Plotting feature importance 

In [ ]:
# print feature importance in descending order of 'total_gain'
total_gain = model.get_booster().get_score(importance_type = 'total_gain')
total_gain_sorted = sorted(total_gain.items(), key=lambda x: x[1], reverse=True)
total_gain_sorted

I could now decide to e.g. to only use features with total_gain > 10k and refit the model with the subset of features that remains

##### 4.3 Please explain your methodology (modeling algorithm/method used and why, what features/data you found useful, what questions you have, and what you would do next with more time)



##### Modeling algorithm/method used and why:
- I used XGBoost as it is the top algorithm when high accuracy is the priority
- Given that fraud is rare I tested model versions with 'increased weight of the minority class'
- I selected the best model version based on maximum net profit 

##### Features/data I found useful
- The most important features were the transactionAmount , whether the card was present and merchantCategoryCode_fuel. 
- The creditLimit, currentBalance and various other merchantCategoryCodes were also important features. 

##### Questions I would have include:
-  e.g. "how are the isfraud labels derived and how reliable are they", "are multi swipes typically flagged as fraud?"....

##### I also tried (but it did not work) to: 

- include the multi swipe result in as a feature in the fraud prediction model
- include the delta between the transaction day and the day of last address change 
- include a feature comparing the card CVV and the entered CVV

##### What I  would do next with more time:
- Look into what fraction of multi swipes were reversed 
- Look into the relationship between multi swipes, reversals and fraud transactions

- I could trial comprehensive hyper parameter tuning
- I could refine the net profit model assumptions
- I could engineer features e.g. feature crosses such as transactionAmount with merchant category 
- I could engineer features e.g.  available funds / credit limit 
- I could engineer features e.g. time of day, day of week, week of year


- I could try a deep autoencoder network model to detect anomalies
- I could try a deep learning classifier with a very large sample and many feature crosses


- I could refine the net profit model by refining the costs of fraud and intervention for different transactionAmount bins and prioritizing which transactionAmount bins to focus the detection modeling effort on. 


- I could engineer cardnumber-specific features such as:
- typical max and mean amount of individual transactionAmount, transactionAmounts per day, transactionAmounts  per week, transactionAmounts per month and the delta of those from the transaction in question
- as above but crossed with e.g. merchant category and perhaps hour of day or day of week

- changes in purchase platform (in person, desktop, mobile) within a short time
- changes in purchase location (large distances between locations) within a short time interval
- a cross of the purchase platform and purchase location change within a short time


- occurrence of multiple online purchases within a short amount of time
- as above by binned by transactionAmounts to catch many small amounts

- WRITE TESTS




